##### Copyright 2023 The TF-Agents Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Búferes de repetición

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/agents/tutorials/5_replay_buffers_tutorial"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/agents/tutorials/5_replay_buffers_tutorial.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/agents/tutorials/5_replay_buffers_tutorial.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a>
</td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/agents/tutorials/5_replay_buffers_tutorial.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar bloc de notas</a></td>
</table>

## Introducción

Los algoritmos de aprendizaje por refuerzo utilizan búferes de repetición para almacenar trayectorias de experiencia al ejecutar una política en un entorno. Durante el entrenamiento, se solicita a los búferes de repetición un subconjunto de las trayectorias (ya sea un subconjunto secuencial o una muestra) para "repetir" la experiencia del agente.

En esta colab, exploramos dos tipos de búferes de repetición: respaldados por Python y respaldados por Tensorflow, que comparten una API común. En las siguientes secciones, describimos la API, cada una de las implementaciones del búfer y cómo usarlas durante el entrenamiento de recopilación de datos.


## Preparación

Si todavía no lo ha hecho, instale tf-agents.

In [ ]:
!pip install tf-agents


In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import numpy as np

from tf_agents import specs
from tf_agents.agents.dqn import dqn_agent
from tf_agents.drivers import dynamic_step_driver
from tf_agents.environments import suite_gym
from tf_agents.environments import tf_py_environment
from tf_agents.networks import q_network
from tf_agents.replay_buffers import py_uniform_replay_buffer
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.specs import tensor_spec
from tf_agents.trajectories import time_step

## API de búfer de repetición

La clase Replay Buffer tiene la siguiente definición y métodos:

```python
class ReplayBuffer(tf.Module):
  """Abstract base class for TF-Agents replay buffer."""

  def __init__(self, data_spec, capacity):
    """Initializes the replay buffer.

    Args:
      data_spec: A spec or a list/tuple/nest of specs describing
        a single item that can be stored in this buffer
      capacity: number of elements that the replay buffer can hold.
    """

  @property
  def data_spec(self):
    """Returns the spec for items in the replay buffer."""

  @property
  def capacity(self):
    """Returns the capacity of the replay buffer."""

  def add_batch(self, items):
    """Adds a batch of items to the replay buffer."""

  def get_next(self,
               sample_batch_size=None,
               num_steps=None,
               time_stacked=True):
    """Returns an item or batch of items from the buffer."""

  def as_dataset(self,
                 sample_batch_size=None,
                 num_steps=None,
                 num_parallel_calls=None):
    """Creates and returns a dataset that returns entries from the buffer."""


  def gather_all(self):
    """Returns all the items in buffer."""
    return self._gather_all()

  def clear(self):
    """Resets the contents of replay buffer"""

```

Tenga en cuenta que cuando se inicializa el objeto búfer de repetición, solicita la `data_spec` de los elementos que almacenará. Esta especificación corresponde a la `TensorSpec` de los elementos de trayectoria que se agregarán al búfer. Esta especificación se adquiere normalmente a partir de la `agent.collect_data_spec` de un agente que define las formas, tipos y estructuras que espera el agente durante el entrenamiento (esto se desarrolla mejor más adelante).

## TFUniformReplayBuffer

`TFUniformReplayBuffer` es el búfer de repetición más usado en TF-Agents, por lo que lo utilizaremos en este tutorial. En `TFUniformReplayBuffer` el almacenamiento del búfer de respaldo se consigue mediante variables tensorflow y, por tanto, forma parte del gráfico de cálculo.

El búfer almacena lotes de elementos y tiene una capacidad máxima de elementos `max_length` por segmento de lote. Por lo tanto, la capacidad total del búfer es `batch_size` x elementos `max_length`. Todos los elementos almacenados en el búfer deben tener una especificación de datos correspondiente. Cuando el búfer de repetición se utiliza para la recopilación de datos, la especificación es la especificación de recopilación de datos del agente.


### Cómo crear el búfer:

Para crear un `TFUniformReplayBuffer` pasamos lo siguiente:

1. la especificación de los elementos de datos que almacenará el búfer
2. el `batch size` correspondiente al tamaño de lote del búfer
3. el número de elementos `max_length` por segmento de lote

A continuación, se muestra un ejemplo de creación de un `TFUniformReplayBuffer` con especificaciones de datos de muestra, `batch_size` 32 y `max_length` 1000.

In [ ]:
data_spec =  (
        tf.TensorSpec([3], tf.float32, 'action'),
        (
            tf.TensorSpec([5], tf.float32, 'lidar'),
            tf.TensorSpec([3, 2], tf.float32, 'camera')
        )
)

batch_size = 32
max_length = 1000

replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    data_spec,
    batch_size=batch_size,
    max_length=max_length)

### Cómo escribir en el búfer:

Para agregar elementos al búfer de repetición, se usa el método `add_batch(items)` donde `items` es una lista, tupla o nido de tensores que representa el lote de elementos que se agregarán al búfer. Cada elemento de `items` debe tener una dimensión exterior igual a `batch_size` y las dimensiones restantes deben ajustarse a las especificaciones de datos del elemento (iguales a las especificaciones de datos pasadas al constructor del búfer de repetición).

Este es un ejemplo de cómo agregar un lote de elementos:


In [ ]:
action = tf.constant(1 * np.ones(
    data_spec[0].shape.as_list(), dtype=np.float32))
lidar = tf.constant(
    2 * np.ones(data_spec[1][0].shape.as_list(), dtype=np.float32))
camera = tf.constant(
    3 * np.ones(data_spec[1][1].shape.as_list(), dtype=np.float32))
  
values = (action, (lidar, camera))
values_batched = tf.nest.map_structure(lambda t: tf.stack([t] * batch_size),
                                       values)
  
replay_buffer.add_batch(values_batched)

### Cómo leer desde el búfer

Hay tres formas de leer datos de `TFUniformReplayBuffer`:

1. `get_next()`: devuelve una muestra del búfer. El tamaño del lote de muestra y el número de pasos de tiempo devueltos se pueden especificar mediante argumentos de este método.
2. `as_dataset()`: devuelve el búfer de repetición como `tf.data.Dataset`. Luego, se puede crear un iterador de conjunto de datos e iterar a través de las muestras de los elementos en el búfer.
3. `gather_all()`: devuelve todos los elementos del búfer como un tensor con forma `[batch, time, data_spec]`

En el siguiente bloque de código se muestran ejemplos de cómo usar cada uno de estos métodos para leer desde el búfer de repetición:

In [ ]:
# add more items to the buffer before reading
for _ in range(5):
  replay_buffer.add_batch(values_batched)

# Get one sample from the replay buffer with batch size 10 and 1 timestep:

sample = replay_buffer.get_next(sample_batch_size=10, num_steps=1)

# Convert the replay buffer to a tf.data.Dataset and iterate through it
dataset = replay_buffer.as_dataset(
    sample_batch_size=4,
    num_steps=2)

iterator = iter(dataset)
print("Iterator trajectories:")
trajectories = []
for _ in range(3):
  t, _ = next(iterator)
  trajectories.append(t)
  
print(tf.nest.map_structure(lambda t: t.shape, trajectories))

# Read all elements in the replay buffer:
trajectories = replay_buffer.gather_all()

print("Trajectories from gather all:")
print(tf.nest.map_structure(lambda t: t.shape, trajectories))


## PyUniformReplayBuffer

`PyUniformReplayBuffer` tiene la misma funcionalidad que `TFUniformReplayBuffer` pero en lugar de variables tf, sus datos se almacenan en arreglos numpy. Este búfer se puede utilizar para la recopilación de datos fuera del gráfico. Tener el almacenamiento de respaldo en numpy puede facilitar la manipulación de datos por parte de algunas aplicaciones (como la indexación para actualizar prioridades) sin usar variables de Tensorflow. Sin embargo, esta implementación no tendrá el beneficio de las optimizaciones de gráficos con Tensorflow.

A continuación, se muestra un ejemplo de cómo crear una instancia de `PyUniformReplayBuffer` a partir de las especificaciones de trayectoria de la política del agente:

In [ ]:
replay_buffer_capacity = 1000*32 # same capacity as the TFUniformReplayBuffer

py_replay_buffer = py_uniform_replay_buffer.PyUniformReplayBuffer(
    capacity=replay_buffer_capacity,
    data_spec=tensor_spec.to_nest_array_spec(data_spec))

## Cómo usar los búferes de repetición durante un entrenamiento

Ahora que sabemos cómo crear un búfer de repetición, escribir elementos en él y leerlos, podemos usarlo para almacenar trayectorias durante el entrenamiento de nuestros agentes.

### Recopilación de datos

En primer lugar, veamos cómo se usa el búfer de repetición durante la recopilación de datos.

En TF-Agents, usamos un `Driver` (consulte el tutorial de controladores para obtener más información) para recopilar experiencia en un entorno. Para usar un `Driver`, especificamos un `Observer` que es una función que el `Driver` debe ejecutar cuando recibe una trayectoria.

Por lo tanto, para agregar elementos de trayectoria al búfer de repetición, agregamos un observador que llama `add_batch(items)` para agregar un lote de elementos en el búfer.

Veamos un ejemplo de esto con `TFUniformReplayBuffer`. En primer lugar, se crean un entorno, una red y un agente. Luego, se crea un `TFUniformReplayBuffer`. Tenga en cuenta que las especificaciones de los elementos de trayectoria en el búfer de repetición son iguales a las especificaciones de recopilación de datos del agente. Luego, se configura su método `add_batch` como observador del controlador que recopilará los datos durante el entrenamiento:


In [ ]:
env = suite_gym.load('CartPole-v0')
tf_env = tf_py_environment.TFPyEnvironment(env)

q_net = q_network.QNetwork(
    tf_env.time_step_spec().observation,
    tf_env.action_spec(),
    fc_layer_params=(100,))

agent = dqn_agent.DqnAgent(
    tf_env.time_step_spec(),
    tf_env.action_spec(),
    q_network=q_net,
    optimizer=tf.compat.v1.train.AdamOptimizer(0.001))

replay_buffer_capacity = 1000

replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    agent.collect_data_spec,
    batch_size=tf_env.batch_size,
    max_length=replay_buffer_capacity)

# Add an observer that adds to the replay buffer:
replay_observer = [replay_buffer.add_batch]

collect_steps_per_iteration = 10
collect_op = dynamic_step_driver.DynamicStepDriver(
  tf_env,
  agent.collect_policy,
  observers=replay_observer,
  num_steps=collect_steps_per_iteration).run()

### Cómo leer los datos para un paso de entrenamiento

Tras agregar elementos de trayectoria al búfer de repetición, podemos leer lotes de trayectorias desde el búfer de repetición para usarlos como datos de entrada para un paso de entrenamiento.

Aquí se muestra un ejemplo de cómo entrenar trayectorias desde el búfer de repetición en un bucle de entrenamiento: 

In [ ]:
# Read the replay buffer as a Dataset,
# read batches of 4 elements, each with 2 timesteps:
dataset = replay_buffer.as_dataset(
    sample_batch_size=4,
    num_steps=2)

iterator = iter(dataset)

num_train_steps = 10

for _ in range(num_train_steps):
  trajectories, _ = next(iterator)
  loss = agent.train(experience=trajectories)
